# Homework 1: Introduction

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt                     

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

import pickle

In [2]:
df = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")

## Q1. Downloading the data

In [3]:
df.shape

(3066766, 19)

## Q2. Computing duration

In [4]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration']  = df['duration'].apply(lambda x: x.total_seconds()/60)
df['duration'].std()

42.59435124195458

## Q3. Dropping outliers

In [5]:
(len(df[(df.duration >= 1) & (df.duration <= 60)])/len(df)) * 100

98.1220282212598

## Q4. One-hot encoding


In [6]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

In [7]:
df[categorical] = df[categorical].astype(str)

In [8]:
train_dict = df[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)
dimensionality = X_train.shape[1]
dimensionality

519

## Q5. Training a model & Q6. Evaluating a model

In [9]:
def read_df(path):
    df = pd.read_parquet(path)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [15]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
target = 'duration'
dv = DictVectorizer()

In [12]:
df_train = read_df('../data/yellow_tripdata_2023-01.parquet')
train_dict = df_train[categorical + numerical].to_dict(orient='records')

X_train = dv.fit_transform(train_dict)
y_train = df_train[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)
print('RMSE on train')
y_pred = lr.predict(X_train)
rms_train = root_mean_squared_error(X_train, y_pred)
print(rms_train)

In [ ]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_val = df_val[target].values



In [11]:
df_val = read_df('../data/yellow_tripdata_2023-02.parquet')

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

train_dict = df_train[categorical + numerical].to_dict(orient='records')
val_dict = df_val[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)
print('RMSE on train')
y_pred_train = lr.predict(X_train)
rms_train = root_mean_squared_error(X_train, y_pred_train)
print(rms_train)

print('RMSE on validation')
y_pred = lr.predict(X_val)
rms = root_mean_squared_error(y_val, y_pred)
print(rms)

RMSE on train


InvalidParameterError: The 'y_true' parameter of root_mean_squared_error must be an array-like. Got <3009173x516 sparse matrix of type '<class 'numpy.float64'>'
	with 9027519 stored elements in Compressed Sparse Row format> instead.